In [1]:
import pandas as pd
import numpy as np

data_click = pd.read_csv('../train_preliminary/click_log.csv')# click_log
data_user = pd.read_csv('../train_preliminary/user.csv')      # user

data_ad = pd.read_csv('../train_preliminary/ad.csv')      # user
data_click = data_click.merge(data_ad,on = 'creative_id',how = 'left')
del data_ad


#对industry广告行业进行特征提取
industry_click = data_click[['user_id', 'product_category', 'click_times']].sort_values(by = 'user_id')
# industry_click = industry_click[data_click['product_category']!='\\N']

industry_click = industry_click.groupby(['user_id','product_category']).agg({'click_times':sum})
industry_click = industry_click.reset_index()


# def func_log(x):
#     return np.log(x+1)
# industry_click['product_category'+'_log'] = industry_click['click_times'].transform(func_log)

# 提取头三个点击最高的种类及点击量
head_x = industry_click.sort_values(['click_times'],ascending=False).groupby(['user_id']).head(3)
head_x = head_x.sort_values('user_id')

del industry_click

def fun1(x):
    x = list(x.values.reshape([-1]))[:6]
    x = x[:6]+[0]*(6-len(x))
    return pd.DataFrame([x])

tops = head_x.groupby('user_id')['product_category','click_times'].apply(fun1)

columns = []
for i in range(6):
    columns.append('product_category'+str(i))
tops.columns = columns
tops = tops.reset_index()
tops = tops.drop(['level_1'],axis = 1)

tops.to_csv('product_category_feat.csv',index=False) 

del tops

/opt/conda/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:35: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


In [ ]:
click_feat = pd.read_csv('clicks_feat.csv')
category_feat = pd.read_csv('category_feat_addlog.csv')
industry_feat = pd.read_csv('industry_feat.csv')
#特征合并在一起
features = click_feat.merge(category_feat,on='user_id',how='left').merge(industry_feat,on='user_id',how='left')
#添加label
user = pd.read_csv('../../data/train_preliminary/user.csv')
data = features.merge(user,on='user_id',how='left')
# del data['user_id']

In [ ]:
# 添加train_cat_max_id_feat特征
def func_cat (x):
    x = x[['category1','category2', 'category3', 'category4', 'category5', 'category6','category7', 'category8', 'category9', 'category10', 'category11','category12', 'category13', 'category14', 'category15', 'category16','category17', 'category18']]
    d = {}
    d['cat_max_id'] = [np.argmax(x.values)+1]
    return pd.DataFrame(d)
cat_max_id_feat = data.groupby('user_id').apply(func_cat)
cat_max_id_feat.reset_index(level=0, inplace=True)
cat_max_id_feat.index = range(len(cat_max_id_feat))
cat_max_id_feat.to_csv('train_cat_max_id_feat.csv',index=False) 


In [ ]:
data = data.merge(cat_max_id_feat,on='user_id',how='left')
data.to_csv('train_feat_user.csv',index=False) 